<a href="https://colab.research.google.com/github/kiankyars/Ultra-Scale-Playbook-Series/blob/main/5_1d_parallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ultra-Scale Playbook: Part 5 - Scaling Data Parallelism and Gradient Accumulation

## Overview
This notebook covers key concepts about scaling LLM training through:
- Understanding the relationship between global batch size, gradient accumulation, and data parallelism
- Calculating optimal configurations for distributed training
- Recognizing the limits of data parallelism
- Practical examples of batch size calculations

## Key Concepts

### The Fundamental Equation
Global batch size (GBS) is determined by:

$$ GBS = MBS \times GA \times DP $$

Where:
- MBS: Maximum Local Batch Size (largest batch that fits on a single GPU)
- GA: Gradient Accumulation Steps
- DP: Data Parallelism (number of GPUs)

### Tradeoffs in Parallelization
We can trade between gradient accumulation (GA) and data parallelism (DP):
- More GA steps = sequential processing (slower but needs fewer GPUs)
- More DP = parallel processing (faster but needs more GPUs)

In practice, maximize DP first since it's inherently parallel, then use GA to reach target batch size.

### Practical Example Setup
1. Determine target global batch size in tokens (from literature/experiments)
2. Find maximum local batch size (MBS) by increasing until GPU memory is full
3. Determine available GPUs for DP
4. Calculate required GA steps to reach target GBS

## Code Implementation

In [ ]:
import torch
from torch import nn
from torch.optim import Adam

# Example model for demonstration
class SimpleModel(nn.Module):
    def __init__(self, input_size=4096, hidden_size=2048):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, input_size)
        
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.output(x)

In [ ]:
def calculate_training_config(target_gbs, mbs, num_gpus):
    """
    Calculate required gradient accumulation steps for target global batch size
    
    Args:
        target_gbs: Desired global batch size
        mbs: Maximum local batch size per GPU
        num_gpus: Number of available GPUs
        
    Returns:
        ga_steps: Required gradient accumulation steps
        effective_gbs: Achieved global batch size
    """
    ga_steps = target_gbs / (mbs * num_gpus)
    
    # GA steps must be at least 1
    if ga_steps < 1:
        print(f"Warning: GA steps {ga_steps} < 1. You have more GPUs than needed.")
        print("Options:")
        print("1. Don't use all GPUs")
        print("2. Increase global batch size")
        print("3. Use smaller MBS for faster processing")
        ga_steps = 1
    
    effective_gbs = mbs * num_gpus * ga_steps
    return ga_steps, effective_gbs

## Interactive Exercises

### Exercise 1: Calculate Training Configuration
Given:
- Target global batch size: 4 million tokens
- Sequence length: 4,000 tokens
- Maximum local batch size: 2 samples per GPU

Calculate:
1. How many GPUs are needed if we want GA steps = 1?
2. If we have 128 GPUs, how many GA steps are needed?

In [ ]:
# Your solution here
target_gbs_tokens = 4_000_000
seq_length = 4000
mbs = 2

# Convert to samples (since MBS is in samples)
target_gbs_samples = target_gbs_tokens / seq_length

# Solution for question 1
ga_steps = 1
required_gpus = target_gbs_samples / (mbs * ga_steps)
print(f"Question 1: Need {required_gpus} GPUs for GA steps = 1")

# Solution for question 2
num_gpus = 128
ga_steps = target_gbs_samples / (mbs * num_gpus)
print(f"Question 2: Need {ga_steps} GA steps with 128 GPUs")

### Exercise 2: Memory Estimation
Estimate the memory requirements for a forward pass of our SimpleModel with:
- Input size: 4096
- Hidden size: 2048
- Batch size: 4
- Precision: float32 (4 bytes per parameter)

In [ ]:
# Your solution here
def estimate_memory(input_size, hidden_size, batch_size):
    # Calculate parameter memory
    params = (input_size * hidden_size) + (hidden_size * hidden_size) + (hidden_size * input_size)
    param_memory = params * 4  # 4 bytes per float32
    
    # Calculate activation memory (simplified estimation)
    # We store input + 2 hidden layer outputs + final output
    activation_memory = (input_size * batch_size + 
                        2 * hidden_size * batch_size + 
                        input_size * batch_size) * 4
    
    total_memory = (param_memory + activation_memory) / (1024 ** 2)  # Convert to MB
    return total_memory

memory_mb = estimate_memory(4096, 2048, 4)
print(f"Estimated memory requirement: {memory_mb:.2f} MB")

## Limits of Data Parallelism
- Communication overhead grows with more GPUs
- Ring latency becomes limiting factor (~512 GPUs with current tech)
- Memory constraints for larger models (even single samples may not fit)

When these limits are hit, we need to explore:
1. Tensor Parallelism (splitting model layers across GPUs)
2. Pipeline Parallelism (splitting model layers sequentially)
3. Fully Sharded Data Parallelism (ZeRO, FSDP)

## Quiz
1. What happens to throughput when you add GPUs beyond the network's capacity?
   a) Increases linearly
   b) Decreases due to communication overhead
   c) Stays constant
   
2. If your GA steps calculation gives 0.5, what does this mean?
   a) You need to double your batch size
   b) You have more GPUs than needed for your target batch size
   c) Your model is too large for the GPUs
   
3. What's the main advantage of data parallelism over gradient accumulation?
   a) It's inherently parallel
   b) It uses less memory
   c) It provides better model accuracy
   
4. What's the first thing to determine when setting up a distributed training run?
   a) Number of available GPUs
   b) Target global batch size
   c) Learning rate

Answers:
1. b) Decreases due to communication overhead
2. b) You have more GPUs than needed for your target batch size
3. a) It's inherently parallel
4. b) Target global batch size

## Summary
- Data parallelism is the first dimension (1D) of parallelization
- Combine DP with gradient accumulation to reach target batch sizes
- There are practical limits to how much DP can help
- Next we'll explore other parallelism techniques (tensor, pipeline, and FSDP)